In [150]:
import pandas as pd 
import numpy as np
import seaborn as sns
pd.set_option('max_columns',200)
pd.set_option('max_rows',2000)

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.ensemble import RandomForestRegressor
#记住这里是在做regression，所以import的是randomforestregressor

from sklearn.metrics import mean_squared_error
from math import sqrt

In [151]:
df_train = pd.read_csv('train_agg.csv',low_memory=False)
df_test = pd.read_csv('test_.csv',low_memory=False)

In [135]:
df_train = df_train.drop(['Unnamed: 0','X','date','fullVisitorId','visitId','visitStartTime','device.isMobile',
                         'geoNetwork.metro','geoNetwork.region','totals.visits','trafficSource.keyword',
                          'trafficSource.isTrueDirect','trafficSource.campaignCode','trafficSource.source'],axis=1)
df_test = df_test.drop(['Unnamed: 0','X','date','fullVisitorId','visitId','visitStartTime','device.isMobile',
                         'geoNetwork.metro','geoNetwork.region','totals.visits','trafficSource.keyword',
                          'trafficSource.isTrueDirect','trafficSource.campaignCode','trafficSource.source'],axis=1)

In [136]:
df_train.head()

,channelGrouping,sessionId,visitNumber,device.browser,device.operatingSystem,device.deviceCategory,geoNetwork.continent,geoNetwork.subContinent,geoNetwork.country,geoNetwork.city,geoNetwork.networkDomain,totals.hits,totals.pageviews,totals.bounces,totals.newVisits,totals.transactionRevenue,trafficSource.campaign,trafficSource.medium,trafficSource.referralPath,trafficSource.adContent,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.isVideoAd
0,Organic Search,3053576296023059465_1472808484,1,Chrome,Macintosh,desktop,Asia,Southeast Asia,Thailand,Bangkok,asianet.co.th,1,1.0,1,1,0,NaN,organic,NaN,NaN,NaN,NaN,NaN,NaN,1
1,Organic Search,0854783508496317255_1472823316,5,Chrome,Android,tablet,Europe,Northern Europe,United Kingdom,NaN,btcentralplus.com,1,1.0,1,0,0,NaN,organic,NaN,NaN,NaN,NaN,NaN,NaN,1
2,Organic Search,1754209820046771603_1472830062,1,Chrome,Macintosh,desktop,Asia,Eastern Asia,China,NaN,ztomy.com,1,1.0,1,1,0,NaN,organic,NaN,NaN,NaN,NaN,NaN,NaN,1
3,Organic Search,5650637137146680834_1472820120,1,Chrome,Windows,desktop,Europe,Western Europe,Switzerland,NaN,roche.com,1,1.0,1,1,0,NaN,organic,NaN,NaN,NaN,NaN,NaN,NaN,1
4,Organic Search,401634289917197318_1472830591,1,Chrome,iOS,mobile,Europe,Western Europe,Netherlands,Amsterdam,kpn.net,1,1.0,1,1,0,NaN,organic,NaN,NaN,NaN,NaN,NaN,NaN,1


### dummy

In [137]:
# label encode the categorical variables and convert the numerical variables to float
cat_cols = ['totals.bounces', 'totals.newVisits',  
                'trafficSource.adwordsClickInfo.isVideoAd',
                'device.deviceCategory','geoNetwork.continent','geoNetwork.subContinent','geoNetwork.country',
           'device.operatingSystem','channelGrouping']


In [138]:
dt= df_train.drop(cat_cols, axis=1)
dtt= df_test.drop(cat_cols, axis=1)

In [139]:
data_train = pd.get_dummies(df_train, columns=cat_cols)

In [140]:
data_test = pd.get_dummies(df_train, columns=cat_cols)

In [141]:
data_train

,sessionId,visitNumber,device.browser,geoNetwork.city,geoNetwork.networkDomain,totals.hits,totals.pageviews,totals.transactionRevenue,trafficSource.campaign,trafficSource.medium,trafficSource.referralPath,trafficSource.adContent,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.adNetworkType,totals.bounces_0,totals.bounces_1,totals.newVisits_0,totals.newVisits_1,trafficSource.adwordsClickInfo.isVideoAd_1,trafficSource.adwordsClickInfo.isVideoAd_False,device.deviceCategory_desktop,device.deviceCategory_mobile,device.deviceCategory_tablet,geoNetwork.continent_Africa,geoNetwork.continent_Americas,geoNetwork.continent_Asia,geoNetwork.continent_Europe,geoNetwork.continent_Oceania,geoNetwork.subContinent_Australasia,geoNetwork.subContinent_Caribbean,geoNetwork.subContinent_Central America,geoNetwork.subContinent_Central Asia,geoNetwork.subContinent_Eastern Africa,geoNetwork.subContinent_Eastern Asia,geoNetwork.subContinent_Eastern Europe,geoNetwork.subContinent_Melanesia,geoNetwork.subContinent_Micronesian Region,geoNetwork.subContinent_Middle Africa,geoNetwork.subContinent_Northern Africa,geoNetwork.subContinent_Northern America,geoNetwork.subContinent_Northern Europe,geoNetwork.subContinent_Polynesia,geoNetwork.subContinent_South America,geoNetwork.subContinent_Southeast Asia,geoNetwork.subContinent_Southern Africa,geoNetwork.subContinent_Southern Asia,geoNetwork.subContinent_Southern Europe,geoNetwork.subContinent_Western Africa,geoNetwork.subContinent_Western Asia,geoNetwork.subContinent_Western Europe,geoNetwork.country_Afghanistan,geoNetwork.country_Albania,geoNetwork.country_Algeria,geoNetwork.country_Andorra,geoNetwork.country_Angola,geoNetwork.country_Antigua & Barbuda,geoNetwork.country_Argentina,geoNetwork.country_Armenia,geoNetwork.country_Aruba,geoNetwork.country_Australia,geoNetwork.country_Austria,geoNetwork.country_Azerbaijan,geoNetwork.country_Bahamas,geoNetwork.country_Bahrain,geoNetwork.country_Bangladesh,geoNetwork.country_Barbados,geoNetwork.country_Belarus,geoNetwork.country_Belgium,geoNetwork.country_Belize,geoNetwork.country_Benin,geoNetwork.country_Bermuda,geoNetwork.country_Bhutan,geoNetwork.country_Bolivia,geoNetwork.country_Bosnia & Herzegovina,geoNetwork.country_Botswana,geoNetwork.country_Brazil,geoNetwork.country_British Virgin Islands,geoNetwork.country_Brunei,geoNetwork.country_Bulgaria,geoNetwork.country_Burkina Faso,geoNetwork.country_Burundi,geoNetwork.country_Cambodia,geoNetwork.country_Cameroon,geoNetwork.country_Canada,geoNetwork.country_Cape Verde,geoNetwork.country_Caribbean Netherlands,geoNetwork.country_Cayman Islands,geoNetwork.country_Central African Republic,geoNetwork.country_Chad,geoNetwork.country_Chile,geoNetwork.country_China,geoNetwork.country_Colombia,geoNetwork.country_Comoros,geoNetwork.country_Congo - Brazzaville,geoNetwork.country_Congo - Kinshasa,geoNetwork.country_Cook Islands,geoNetwork.country_Costa Rica,geoNetwork.country_Croatia,...,geoNetwork.country_Nepal,geoNetwork.country_Netherlands,geoNetwork.country_New Caledonia,geoNetwork.country_New Zealand,geoNetwork.country_Nicaragua,geoNetwork.country_Niger,geoNetwork.country_Nigeria,geoNetwork.country_Northern Mariana Islands,geoNetwork.country_Norway,geoNetwork.country_Oman,geoNetwork.country_Pakistan,geoNetwork.country_Palestine,geoNetwork.country_Panama,geoNetwork.country_Papua New Guinea,geoNetwork.country_Paraguay,geoNetwork.country_Peru,geoNetwork.country_Philippines,geoNetwork.country_Poland,geoNetwork.country_Portugal,geoNetwork.country_Puerto Rico,geoNetwork.country_Qatar,geoNetwork.country_Romania,geoNetwork.country_Russia,geoNetwork.country_Rwanda,geoNetwork.country_Réunion,geoNetwork.country_San Marino,geoNetwork.country_Saudi Arabia,geoNetwork.country_Senegal,geoNetwork.country_Serbia,geoNetwork.country_Seychelles,geoNetwork.country_Sierra Leone,geoNetwork.country_Singapore,geoNetwork.country_Sint Maarten,geoNetwork.countr

In [142]:
data_train = data_train.drop(['device.browser','geoNetwork.city','geoNetwork.networkDomain','trafficSource.campaign',
                'trafficSource.medium','trafficSource.referralPath','trafficSource.adContent','trafficSource.adwordsClickInfo.slot',
                'trafficSource.adwordsClickInfo.page','trafficSource.adwordsClickInfo.gclId','trafficSource.adwordsClickInfo.adNetworkType'],
                axis=1)
data_test=data_test.drop(['device.browser','geoNetwork.city','geoNetwork.networkDomain','trafficSource.campaign',
                'trafficSource.medium','trafficSource.referralPath','trafficSource.adContent','trafficSource.adwordsClickInfo.slot',
                'trafficSource.adwordsClickInfo.page','trafficSource.adwordsClickInfo.gclId','trafficSource.adwordsClickInfo.adNetworkType'],
                axis=1)

In [143]:
data_train['totals.pageviews'] = data_train['totals.pageviews'].replace(np.nan,0)
data_test['totals.pageviews'] = data_test['totals.pageviews'].replace(np.nan,0)

In [144]:
X_train = data_train.drop(['totals.transactionRevenue','sessionId'],axis=1)

feature_list = list(X_train.columns)

X_train = X_train.values
y_train=data_train['totals.transactionRevenue'].values 
print(X_train.shape,y_train.shape)

(402260, 277) (402260,)


In [106]:
data_test

,sessionId,visitNumber,totals.hits,totals.pageviews,totals.transactionRevenue,totals.bounces_0,totals.bounces_1,totals.newVisits_0,totals.newVisits_1,trafficSource.adwordsClickInfo.isVideoAd_1,trafficSource.adwordsClickInfo.isVideoAd_False,device.deviceCategory_desktop,device.deviceCategory_mobile,device.deviceCategory_tablet,geoNetwork.continent_Africa,geoNetwork.continent_Americas,geoNetwork.continent_Asia,geoNetwork.continent_Europe,geoNetwork.continent_Oceania,geoNetwork.subContinent_Australasia,geoNetwork.subContinent_Caribbean,geoNetwork.subContinent_Central America,geoNetwork.subContinent_Central Asia,geoNetwork.subContinent_Eastern Africa,geoNetwork.subContinent_Eastern Asia,geoNetwork.subContinent_Eastern Europe,geoNetwork.subContinent_Melanesia,geoNetwork.subContinent_Micronesian Region,geoNetwork.subContinent_Middle Africa,geoNetwork.subContinent_Northern Africa,geoNetwork.subContinent_Northern America,geoNetwork.subContinent_Northern Europe,geoNetwork.subContinent_Polynesia,geoNetwork.subContinent_South America,geoNetwork.subContinent_Southeast Asia,geoNetwork.subContinent_Southern Africa,geoNetwork.subContinent_Southern Asia,geoNetwork.subContinent_Southern Europe,geoNetwork.subContinent_Western Africa,geoNetwork.subContinent_Western Asia,geoNetwork.subContinent_Western Europe,geoNetwork.country_Afghanistan,geoNetwork.country_Albania,geoNetwork.country_Algeria,geoNetwork.country_Andorra,geoNetwork.country_Angola,geoNetwork.country_Antigua & Barbuda,geoNetwork.country_Argentina,geoNetwork.country_Armenia,geoNetwork.country_Aruba,geoNetwork.country_Australia,geoNetwork.country_Austria,geoNetwork.country_Azerbaijan,geoNetwork.country_Bahamas,geoNetwork.country_Bahrain,geoNetwork.country_Bangladesh,geoNetwork.country_Barbados,geoNetwork.country_Belarus,geoNetwork.country_Belgium,geoNetwork.country_Belize,geoNetwork.country_Benin,geoNetwork.country_Bermuda,geoNetwork.country_Bhutan,geoNetwork.country_Bolivia,geoNetwork.country_Bosnia & Herzegovina,geoNetwork.country_Botswana,geoNetwork.country_Brazil,geoNetwork.country_British Virgin Islands,geoNetwork.country_Brunei,geoNetwork.country_Bulgaria,geoNetwork.country_Burkina Faso,geoNetwork.country_Burundi,geoNetwork.country_Cambodia,geoNetwork.country_Cameroon,geoNetwork.country_Canada,geoNetwork.country_Cape Verde,geoNetwork.country_Caribbean Netherlands,geoNetwork.country_Cayman Islands,geoNetwork.country_Central African Republic,geoNetwork.country_Chad,geoNetwork.country_Chile,geoNetwork.country_China,geoNetwork.country_Colombia,geoNetwork.country_Comoros,geoNetwork.country_Congo - Brazzaville,geoNetwork.country_Congo - Kinshasa,geoNetwork.country_Cook Islands,geoNetwork.country_Costa Rica,geoNetwork.country_Croatia,geoNetwork.country_Curaçao,geoNetwork.country_Cyprus,geoNetwork.country_Czechia,geoNetwork.country_Côte d’Ivoire,geoNetwork.country_Denmark,geoNetwork.country_Djibouti,geoNetwork.country_Dominica,geoNetwork.country_Dominican Republic,geoNetwork.country_Ecuador,geoNetwork.country_Egypt,geoNetwork.country_El Salvador,...,geoNetwork.country_Nepal,geoNetwork.country_Netherlands,geoNetwork.country_New Caledonia,geoNetwork.country_New Zealand,geoNetwork.country_Nicaragua,geoNetwork.country_Niger,geoNetwork.country_Nigeria,geoNetwork.country_Northern Mariana Islands,geoNetwork.country_Norway,geoNetwork.country_Oman,geoNetwork.country_Pakistan,geoNetwork.country_Palestine,geoNetwork.country_Panama,geoNetwork.country_Papua New Guinea,geoNetwork.country_Paraguay,geoNetwork.country_Peru,geoNetwork.country_Philippines,geoNetwork.country_Poland,geoNetwork.country_Portugal,geoNetwork.country_Puerto Rico,geoNetwork.country_Qatar,geoNetwork.country_Romania,geoNetwork.country_Russia,geoNetwork.country_Rwanda,geoNetwork.country_Réunion,geoNetwork.country_San Marino,geoNetwork.country_Saudi Arabia,geoNetwork.country_Senegal,geoNetwork.country_Serbia,geoNetwork.country_Seychelles,geoNetwork.country_Sierra Leone,geoNetwork.country_Singapore,geoNetwork.country_Sint Maarten,geoNe

In [145]:
X_test = data_test.drop(['totals.transactionRevenue','sessionId'],axis=1)
X_test = X_test.values

y_test=data_test['totals.transactionRevenue'].values 
y_test.shape

print(X_test.shape,y_test.shape)

(402260, 277) (402260,)


In [146]:
#建模
rf = RandomForestRegressor(n_estimators=1000,random_state=0)

#训练
rf.fit(X_train,y_train)

#prediction
prediction_test = rf.predict(X_test)

In [147]:
rmse_test = sqrt(mean_squared_error(y_test, prediction_test))

print('rmse - test: ',rmse_test)

rmse - test:  17898020.158905644


In [148]:
# get the feature importance
importances = list(rf.feature_importances_)

# transform the formula
feature_importances = [(feature, importance) for feature, importance in zip(feature_list, importances)]

# descending order
feature_importances = sorted(feature_importances, key= lambda x:x[1], reverse=True)

# print out
for a in feature_importances:
    print('Variable: {}      Importance:{}'.format(a[0],a[1]))


Variable: totals.pageviews      Importance:0.3166344754335812
Variable: visitNumber      Importance:0.297500611178096
Variable: totals.hits      Importance:0.2514998481714107
Variable: channelGrouping_Direct      Importance:0.06869061254773248
Variable: device.operatingSystem_Windows      Importance:0.0207265669451592
Variable: device.operatingSystem_Macintosh      Importance:0.009109209637820095
Variable: device.operatingSystem_Chrome OS      Importance:0.005606210549752617
Variable: channelGrouping_Organic Search      Importance:0.005184713769110332
Variable: channelGrouping_Referral      Importance:0.0051689097447210125
Variable: totals.newVisits_0      Importance:0.0038927789540089968
Variable: totals.newVisits_1      Importance:0.0035399954004772705
Variable: geoNetwork.country_Canada      Importance:0.002740410999604017
Variable: geoNetwork.country_United States      Importance:0.0023026656456169256
Variable: channelGrouping_Display      Importance:0.0008388686801489511
Variable:

In [ ]:
# order the features
sorted_importances = [importance[1] for importance in feature_importances]
sorted_features = [importance[0] for importance in feature_importances]

# cumulative_importances
cumulative_importances = np.cumsum(sorted_importances)

cumulative_importances

In [ ]:
sorted_features